In [2]:
import numpy as np
from astropy.io import fits

In [29]:
def findpetrorad(filename, rowc=100, colc=100, FITS_directory='Downloads/allison/', rpinit=3):
    #Finds the petrosian radius as defined here: http://skyserver.sdss.org/dr7/en/help/docs/algorithm.asp?key=mag_petro
    #Initialise rp at 1 pixel. Calculate petrosian ratio. If not less than 0.2, increase rp and try again
    rp=rpinit
    file=fits.open(FITS_directory+filename)
    image=file[0] 
    data=image.data
    while True:
        top=2*(1.25**2-0.8**2)**-1*rp**-2*np.trapz([r*azimuthal_averaged_brightness(data, r, rowc, colc) for r in np.arange(0.8*rp,1.25*rp, 1)], np.arange(0.8*rp,1.25*rp, 1))
        bottom=2*rp**-2*np.trapz([r*azimuthal_averaged_brightness(data, r,colc, rowc) for r in np.arange(0,rp, 1)], np.arange(0,rp,1))
        if top/bottom<0.2:
            return rp
        else:
            rp+=1
            continue
def azimuthal_averaged_brightness(image, radius, colc, rowc):
    #Finds the azimuthally averaged intensity at a particular radius
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    thetarad=[i*2*np.pi/360. for i in thetas]
    pixbrightness=[]
    for i in range(len(thetarad)):
        #First calculate distances from centre
        dx=radius*np.sin(thetarad[i])
        dy=radius*np.cos(thetarad[i])
        #Then add on coordinates of centre
        x=int(round(colc+dx))
        y=int(round(rowc+dy))
        pixbrightness.append(image[y, x])
    average=(2*np.pi)**-1*np.trapz(pixbrightness, thetarad)
    return average

In [49]:
print(findpetrorad('imrun708dwfDiffLF.00240.pa0.inc0.fits'))
print(findpetrorad('imrun708dwfDiffLF.00250.pa0.inc0.fits'))


8
9
